In [1]:
import os
from langchain_redis import RedisVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from dotenv import load_dotenv

load_dotenv()



True

In [2]:
## 키워드로 장기/단기로 나누기

import os
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain_redis import RedisVectorStore
from langchain.docstore.document import Document
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 가져오기
REDIS_URL = os.getenv('REDIS_URL')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# OpenAI Embeddings 설정
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# RedisVectorStore 설정 (embeddings 객체를 직접 전달)
redis_vector_store = RedisVectorStore(redis_url=REDIS_URL, embeddings=embeddings)

# OpenAI LLM 설정
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

# 문서를 Redis에 저장하는 함수
def save_document_to_redis(text, metadata=None):
    # 문서 생성 (metadata를 빈 딕셔너리로 전달)
    if metadata is None:
        metadata = {}
    document = Document(page_content=text, metadata=metadata)
    redis_vector_store.add_documents([document])
    print(f"문서 저장됨: {text}...")  # 첫 50자만 출력하여 확인

# Query Routing: 사용자의 쿼리에 따라 적절한 저장소에서 응답을 생성하는 함수
def query_routing(query):
    retriever = redis_vector_store.as_retriever()

    # RetrievalQA 체인 설정
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever
    )

    # 질의에 대한 답변 가져오기
    response = qa_chain.run(query)
    print(f"질문: {query}")
    print(f"답변: {response}")  # 답변을 출력
    return response

# 사용자 메시지에 따라 장기기억을 사용할지 판단하는 함수
def should_use_long_term_memory(query):
    # 예시: 사용자가 "오대산" 또는 "산신령" 관련 질문을 했다면 장기기억을 사용하도록 설정
    if "오대산" in query or "산신령" in query:
        return True
    return False

# 사용자와의 대화 예시: 기억을 기반으로 질문과 답변을 이어나가도록 설계
def long_term_memory_conversation(queries):
    for query in queries:
        print(f"\n질문: {query}")
        
        # 사용자가 장기기억을 사용해야 하는지 판단
        if should_use_long_term_memory(query):
            print("🍳장기기억을 사용하여 질문에 답변합니다.")
            response = query_routing(query)  # Redis에서 쿼리 검색
        else:
            print("🍕장기기억을 사용하지 않고 새로 처리합니다.")
            response = f"새로운 질문: '{query}'에 대한 답변입니다."
        
        # 답변을 저장
        save_document_to_redis(f"사용자가 묻은 질문: {query}. 답변: {response}")

# 예시 대화: 첫 번째와 두 번째 질문
queries = [
    "오대산의 산신령은 홍주연이래",  # 장기기억 사용
    "오대산의 산신령이 누구라고?",  # 장기기억 사용
    "오늘 점심은 김치짬뽕이야",  # 장기기억 미사용
    "오늘 점심은 뭐라고?"
]

long_term_memory_conversation(queries)



질문: 오대산의 산신령은 홍주연이래
🍳장기기억을 사용하여 질문에 답변합니다.


C:\Users\20111\AppData\Local\Temp\ipykernel_4044\2840789469.py:47: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


질문: 오대산의 산신령은 홍주연이래
답변:  I'm sorry, I don't know who 홍주연 is or what the context of this sentence is.
문서 저장됨: 사용자가 묻은 질문: 오대산의 산신령은 홍주연이래. 답변:  I'm sorry, I don't know who 홍주연 is or what the context of this sentence is....

질문: 오대산의 산신령이 누구라고?
🍳장기기억을 사용하여 질문에 답변합니다.
질문: 오대산의 산신령이 누구라고?
답변:  I don't know, but it seems like 홍주연 might have some connection to the 산신령 at 오대산.
문서 저장됨: 사용자가 묻은 질문: 오대산의 산신령이 누구라고?. 답변:  I don't know, but it seems like 홍주연 might have some connection to the 산신령 at 오대산....

질문: 오늘 점심은 김치짬뽕이야
🍕장기기억을 사용하지 않고 새로 처리합니다.
문서 저장됨: 사용자가 묻은 질문: 오늘 점심은 김치짬뽕이야. 답변: 새로운 질문: '오늘 점심은 김치짬뽕이야'에 대한 답변입니다....

질문: 오늘 점심은 뭐라고?
🍕장기기억을 사용하지 않고 새로 처리합니다.
문서 저장됨: 사용자가 묻은 질문: 오늘 점심은 뭐라고?. 답변: 새로운 질문: '오늘 점심은 뭐라고?'에 대한 답변입니다....


In [74]:
## 별 의미 없는거 물어보기

queries = [
    "오늘 점심은 김치짬뽕이야",  # 장기기억 미사용
    "오늘 점심은 뭐라고?"
]

In [3]:
import redis
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

# Redis 서버의 비밀번호를 .env 파일에서 불러오기
REDIS_PASSWORD = os.getenv('REDIS_PASSWORD')

# Redis 클라이언트 설정 (비밀번호 추가)
r = redis.Redis(host='localhost', port=6379, db=0, password=REDIS_PASSWORD)

# 저장된 키 확인
keys = r.keys('*')  # 모든 키 조회
print("모든 키:", keys)


모든 키: [b'01JDH7ATCJ114T57YVG0QN8V01:5ff427c18d2a4fe688d8bce5f126e295', b'01JDH7CDS87703WQ4WBD36ECYF:cb4f7bdc1e5f464bb5fe576522fe9313', b'01JDH7CDS87703WQ4WBD36ECYF:bd364d364bda458d80867617eb26557f', b'01JDH7CDS87703WQ4WBD36ECYF:070ac49ff7d94c30a4a5e5233c01e234', b'01JDH7CDS87703WQ4WBD36ECYF:f5c32374ed30493b985097b39589cf93']


In [4]:
import redis
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

# Redis 서버의 비밀번호를 .env 파일에서 불러오기
REDIS_PASSWORD = os.getenv('REDIS_PASSWORD')

# Redis 클라이언트 설정 (비밀번호 추가)
r = redis.Redis(host='localhost', port=6379, db=0, password=REDIS_PASSWORD)

# 전체 데이터 삭제
r.flushdb()
print("전체 Redis 데이터가 삭제되었습니다.")

# 특정 데이터 삭제
# r.delete('long_term_memory')
# print("특정 키('long_term_memory')가 삭제되었습니다.")



전체 Redis 데이터가 삭제되었습니다.
